# 7.1. 深度卷积神经网络（AlexNet）

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils import data
import torch.optim as optim

EPOCH = 10
batchsize = 32
LR=0.01
transform = transforms.Compose([transforms.ToTensor()])
train_datasets = datasets.FashionMNIST(root='./data', train=True, transform=transform)
test_datasets = datasets.FashionMNIST(root='./data', train=False, transform=transform)

train_dataloader = data.DataLoader(train_datasets, batch_size=batchsize, shuffle=True)
test_dataloader = data.DataLoader(test_datasets, batch_size=batchsize, shuffle=False)

net = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d((3, 3)),
    nn.Flatten(),
    nn.Linear(256 * 3 * 3, 1024), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 10)
)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LR)

for i in range(EPOCH):
    total_loss, correct, total = 0, 0, 0
    net.train()
    for X, y in train_dataloader:
        o = net(X)
        l = loss(o, y)

        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        total_loss += l.item()
        correct += (o.argmax(1) == y).sum().item()
        total += y.size(0)

    train_acc = correct / total
    print(f"Epoch [{i+1}/{EPOCH}]  Loss: {total_loss/len(train_dataloader):.4f}  Accuracy: {train_acc:.4f}")
        



Epoch [1/10]  Loss: 0.5957  Accuracy: 0.7882
Epoch [2/10]  Loss: 0.4430  Accuracy: 0.8397
Epoch [3/10]  Loss: 0.4337  Accuracy: 0.8429
Epoch [4/10]  Loss: 0.4144  Accuracy: 0.8500
Epoch [5/10]  Loss: 0.4165  Accuracy: 0.8483
Epoch [6/10]  Loss: 0.4150  Accuracy: 0.8475
Epoch [7/10]  Loss: 0.4088  Accuracy: 0.8538
Epoch [8/10]  Loss: 0.4094  Accuracy: 0.8521
Epoch [9/10]  Loss: 0.4076  Accuracy: 0.8533
Epoch [10/10]  Loss: 0.4059  Accuracy: 0.8541


In [28]:
net.eval() 
total_loss, correct, total = 0, 0, 0
for X, y in test_dataloader:
    o = net(X)
    correct += (o.argmax(1) == y).sum().item()
    total += y.size(0)

train_acc = correct / total
print(f"Accuracy: {train_acc:.4f}")
    

Accuracy: 0.8484


# 7.2. 使用块的网络（VGG）

In [31]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        nn.Sequential(*conv_blks), nn.Flatten(),
        # 全连接层部分
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
net = vgg(conv_arch)

# 7.3 网络中的网络（NiN）

In [32]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])
